# ***Demo: Amazon SageMaker Script Mode and Custom Container***

In [1]:
# !pip install sagemaker --upgrade -q
# !pip install ipywidgets -q
# !wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
# !python generate_cifar10_tfrecords.py --data-dir cifar10

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [2]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

#### Upload dataset if it doesn't exist

In [3]:
# datasets = sagemaker_session.upload_data(path='cifar10', key_prefix=f'{dataset_folder}/cifar10-dataset')
# datasets

#If dataset already exists
datasets = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset'
datasets

's3://sagemaker-us-west-2-453691756499/datasets/cifar10-dataset'

# SageMaker training with scrip mode

![](./sm_script_mode.png)

In [4]:
job_name   = f'tf-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

metric_definitions = [{'Name': 'Validation Accuracy', 'Regex': 'Validation Accuracy: ([0-9\\.]+)'}]

hyperparameters = {'epochs'       : 50, 
                   'learning-rate': 0.01,
                   'momentum'     : 0.95,
                   'weight-decay' : 2e-4,
                   'optimizer'    : 'adam',
                   'batch-size'   : 256,
                   'model-type'   : 'custom'}


In [5]:
from sagemaker.debugger import TensorBoardOutputConfig

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{bucket_name}/tensorboard_logs')

In [6]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(entry_point         = 'cifar10-tf2.py', 
                           source_dir           = 'code',
                           output_path          = output_path + '/',
                           code_location        = output_path,
                           role                 = role,
                           instance_count       = 1,
                           instance_type        = 'ml.p3.2xlarge',
                           framework_version    = '2.4', 
                           py_version           = 'py37',
                           metric_definitions   = metric_definitions,
                           hyperparameters      = hyperparameters,
                           tensorboard_output_config = tensorboard_output_config)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [7]:
estimator.fit({'train': datasets,
                    'validation': datasets,
                    'eval': datasets}, 
                  job_name=job_name, wait=True)

2021-05-11 09:43:04 Starting - Starting the training job...
2021-05-11 09:43:27 Starting - Launching requested ML instancesProfilerReport-1620726184: InProgress
.........
2021-05-11 09:44:48 Starting - Preparing the instances for training......
2021-05-11 09:45:55 Downloading - Downloading input data...
2021-05-11 09:46:28 Training - Downloading the training image....................2021-05-11 09:49:42.284639: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-05-11 09:49:42.290887: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-11 09:49:42.396253: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-11 09:49:42.503019: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profi

# SageMaker training with custom container

![](sm_custom_container.png)

In [8]:
client = boto3.client('sagemaker')
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

repo_name = 'eks-ml-containers'
image_tag = 'latest-gpu'

image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, repo_name, image_tag)

In [9]:
from sagemaker.estimator import Estimator
job_name = f'tf-single-gpu-custom-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'

estimator = Estimator(
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc:([0-9\\.]+)'}])

estimator.fit(job_name=job_name)

2021-05-11 09:53:50 Starting - Starting the training job...
2021-05-11 09:54:14 Starting - Launching requested ML instancesProfilerReport-1620726830: InProgress
......
2021-05-11 09:55:14 Starting - Preparing the instances for training......
2021-05-11 09:56:17 Downloading - Downloading input data...
2021-05-11 09:56:34 Training - Downloading the training image...........................2021-05-11 10:01:08.070478: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-05-11 10:01:08.076179: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-11 10:01:08.175098: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-11 10:01:08.275514: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker P